In [1]:
## This script uses df_ols, so it needs to be run after 4_citations/0_data_prep.ipynb

In [2]:
%config Completer.use_jedi = False

In [3]:
import pandas as pd
import pickle
# import pickle5 as pickle
from tqdm.notebook import tqdm
import numpy as np
import os
import glob

In [4]:
def save(x, file_name):
    with open(file_name, 'wb') as handle:
        pickle.dump(x, handle, protocol=pickle.HIGHEST_PROTOCOL)

def restore(file_name):
    with open(file_name, 'rb') as handle:
        x = pickle.load(handle)
    return x

In [5]:
df = pd.read_pickle('../../data/df_ols.p')

In [6]:
rg_cols = ['white_M', 'hispanic_M', 'black_M', 'asian_M','white_F', 'hispanic_F', 'black_F', 'asian_F']
topics_col = ['topic_'+ str(x) for x in range(1,201)]
inst_cols = ['avg_citations_Q','avg_citations_Q10','usnr_rank_cat','selindex','hbcu','hsi','womens']

In [7]:
path = '/data/datasets/WOS/US/lda_fields/'
doc_dist_files = os.listdir(path)

li = []
for file in doc_dist_files:
    _ = pd.read_pickle(path+file)
    li.append(_)
    
doc_dist = pd.concat(li, axis=0, ignore_index=True)

In [8]:
#df = df.merge(doc_dist, on ='id_art')

In [9]:
df_cm = df[df.EDiscipline == 'Clinical Medicine']

In [10]:
avg_cit_topic = (df_cm.filter(like='topic').T@df_cm['cit_rel_all_IAC'])/df_cm.filter(like='topic').sum()

In [11]:
disciplines = df.EDiscipline.unique()
university_groups_prop = pd.DataFrame()
for discipline in tqdm(disciplines):
    university_groups_prop_ = pd.DataFrame()
    df_d = df[df.EDiscipline == discipline]
    avg_cit_topic_d = (df_d.filter(like='topic').T@df_d['cit_rel_all_IAC'])/df_d.filter(like='topic').sum()
    avg_cit_topic_d['EDiscipline'] = discipline
    avg_cit_topic_d['inst_group'] = 'citations'
    avg_cit_topic_d['inst_tag'] = 'citations'
    for inst in inst_cols:
        df_group = df_d.groupby(inst)[topics_col].sum()
        university_groups_prop_topic = df_group.div(df_group.sum(axis=0), axis=1).reset_index(level=inst)
        university_groups_prop_ = university_groups_prop_.append(university_groups_prop_topic)
    university_groups_prop_ = university_groups_prop_.melt(id_vars=topics_col,var_name='inst_group',value_name='inst_tag').dropna()
    university_groups_prop_['EDiscipline'] = discipline
    university_groups_prop_ = university_groups_prop_.append(avg_cit_topic_d,ignore_index=True )
    university_groups_prop = university_groups_prop.append(university_groups_prop_)



  0%|          | 0/13 [00:00<?, ?it/s]

/tmp/ipykernel_1711/54057086.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  university_groups_prop_ = university_groups_prop_.append(university_groups_prop_topic)
/tmp/ipykernel_1711/54057086.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  university_groups_prop_ = university_groups_prop_.append(university_groups_prop_topic)
/tmp/ipykernel_1711/54057086.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  university_groups_prop_ = university_groups_prop_.append(university_groups_prop_topic)
/tmp/ipykernel_1711/54057086.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  university_groups_prop_ = university_groups_prop_.appen

In [12]:
#all fields but SSH and profesional fields have 200 topics, and SSH and profesional fields have 300 

In [13]:
university_groups_prop[university_groups_prop.inst_group=='citations']

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,...,topic_291,topic_292,topic_293,topic_294,topic_295,topic_296,topic_297,topic_298,topic_299,topic_300
26,1.369687,1.268345,1.315075,1.392846,1.361381,1.172233,1.370142,1.533291,1.260004,1.182980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,1.339658,1.455655,1.481806,1.256225,1.236302,1.311825,1.405130,1.726170,1.452373,1.528178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,1.400874,1.682468,1.680907,1.569241,1.420320,1.505062,1.785534,1.375589,2.506400,1.361144,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,1.467451,1.871764,1.471583,1.356824,1.293023,1.293906,1.416381,1.446132,1.539188,1.494380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,1.172789,1.151866,1.127454,1.391147,1.087750,1.172769,1.466405,1.260001,1.215002,1.660200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,1.322919,1.430659,1.248083,1.154128,1.054539,1.291700,1.610245,1.333528,1.200508,1.547923,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,1.188908,1.136653,1.266192,1.410264,1.152355,1.267171,1.180564,1.144598,1.142454,1.194646,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,1.784193,1.307540,1.260036,1.303749,1.220638,1.205852,1.111992,1.155784,1.255574,1.243391,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,1.193445,1.656789,1.402240,1.254541,1.268302,1.171918,1.644076,1.208520,1.239169,1.053005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,1.347625,1.368996,1.477036,1.444540,1.512512,1.374295,1.412581,1.320110,1.270943,1.371576,...,1.194201,1.491065,1.375774,1.514892,1.606591,1.542644,1.272436,1.337655,1.477528,1.403251


In [14]:
university_groups_prop.inst_group.value_counts()

avg_citations_Q10    130
avg_citations_Q       39
usnr_rank_cat         39
selindex              39
womens                39
hbcu                  26
hsi                   26
citations             13
Name: inst_group, dtype: int64

In [15]:
university_groups_prop.to_csv('../../results/institutions_topics/topic_prop_citations_by_inst_all_disciplines.csv', index=False)